<a href="https://colab.research.google.com/github/LCaravaggio/politext/blob/snowball/Parseo_de_OrdenesdelDia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d leonardocaravaggio/scraps-pdfs -f "Scraps PDFs.csv"
!kaggle datasets download -d leonardocaravaggio/scraps-pdfs -f "base_formateada_senado_snowball.csv"

Mounted at /content/drive
 97% 227M/234M [00:03<00:00, 65.1MB/s]
100% 234M/234M [00:03<00:00, 69.9MB/s]
 90% 129M/143M [00:01<00:00, 90.9MB/s]
100% 143M/143M [00:01<00:00, 80.4MB/s]


In [2]:
import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [3]:
# Cargamos la base
base = pd.read_csv('/content/Scraps PDFs.csv')
base_parseada = pd.read_csv('/content/base_formateada_senado_snowball.csv')

<ipython-input-3-e0eeee1141a8>:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  base_parseada = pd.read_csv('/content/base_formateada_senado_snowball.csv')


In [212]:
base_parseada.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Nombre/Cargo', 'Discurso', 'Cargo',
       'Nombre', 'Link', 'Legislatura', 'tipo', 'fecha', 'fecha_formato',
       'nombre', 'apellidos', 'procedLiteral', 'procedLugar', 'grupoNombre',
       'tokens'],
      dtype='object')

In [260]:
import re

def generar(l):
  df=pd.DataFrame(columns=['Nombre/Cargo', 'Discurso', 'Cargo',
       'Nombre', 'Link', 'Legislatura', 'tipo', 'fecha', 'fecha_formato',
       'nombre', 'apellidos', 'procedLiteral', 'procedLugar', 'grupoNombre',
       'tokens', 'Orden'])
  fila=base['link'][base['link'] == l[6]].index[0]
  texto=str(base.loc[fila].text)
  discurso=str(l[3])

  patron = r'\b[A-Z][a-z][^.!?]*[.!?]'
  ordenes = re.findall(patron, texto[texto.find('ORDEN DEL DÍA'):texto.find('SUMARIO')].replace('\n',''))


  o=0
  for orden in ordenes:
    if texto[texto.find('SUMARIO'):].find(discurso[:25])>texto[texto.find('SUMARIO'):].find(orden.upper()): o+=1

  df['Nombre/Cargo']=[l[2]]
  df['Discurso']=[l[3]]
  df['Cargo']=[l[4]]
  df['Nombre']=[l[5]]
  df['Link']=[l[6]]
  df['Legislatura']=[l[7]]
  df['tipo']=[l[8]]
  df['fecha']=[l[9]]
  df['fecha_formato']=[l[10]]
  df['nombre']=[l[11]]
  df['apellidos']=[l[12]]
  df['procedLiteral']=[l[13]]
  df['procedLugar']=[l[14]]
  df['grupoNombre']=[l[15]]
  df['tokens']=[l[16]]
  try:
    df['orden']=[ordenes[o-1]]
  except:
    df['orden']=[""]


  return df

In [261]:
base2=pd.DataFrame(columns=['Nombre/Cargo', 'Discurso', 'Cargo',
       'Nombre', 'Link', 'Legislatura', 'tipo', 'fecha', 'fecha_formato',
       'nombre', 'apellidos', 'procedLiteral', 'procedLugar', 'grupoNombre',
       'tokens', 'Orden'])

In [ ]:
for row in base_parseada.values:
  base2 = pd.concat([base2, generar(row)], axis=0)

In [ ]:
base2=base2.reset_index(drop=True)

In [ ]:
base2['fecha_formato'] = pd.to_datetime(base2['fecha_formato'], errors='coerce')

In [ ]:
from google.colab import files
base2.to_csv('base_formateada_senado_snowball_conorden.csv', escapechar='\\')
files.download('base_formateada_senado_snowball_conorden.csv')